In [1]:
# from posthook import execute_posthook

!python main.py

latest_date

c:\Users\user\OneDrive\Documents\SE_Factory\FSD\Final_Project\usa_recession_analysis\database_handler.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)
c:\Users\user\OneDrive\Documents\SE_Factory\FSD\Final_Project\usa_recession_analysis\database_handler.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)



2023-10-09 06:32:00


In [19]:
from politicians_speeches_and_social_media_posts import get_politician_social_media_posts
from database_handler import create_connection
get_politician_social_media_posts(create_connection())

           _golden _unit_state  _trusted_judgments   _last_judgment_at  \
_unit_id                                                                 
766192484    False   finalized                   1 2015-08-04 21:17:00   
766192485    False   finalized                   1 2015-08-04 21:20:00   
766192486    False   finalized                   1 2015-08-04 21:14:00   
766192487    False   finalized                   1 2015-08-04 21:08:00   
766192488    False   finalized                   1 2015-08-04 21:26:00   
...            ...         ...                 ...                 ...   
766197482    False   finalized                   1 2015-08-05 06:05:00   
766197483    False   finalized                   1 2015-08-05 05:57:00   
766197484    False   finalized                   1 2015-08-05 05:10:00   
766197485    False   finalized                   1 2015-08-05 06:08:00   
766197486    False   finalized                   1 2015-08-05 05:04:00   

           audience  audience:confide

In [ ]:
from yahoofinancials import YahooFinancials
import pandas as pd
from datetime import datetime, date, time,timedelta
import pytz

# tickers = ['META','AMZN','AAPL','NFLX','GOOGL']
tickers = ['AMZN']

start_date = date(2023,10,11)
start_date = start_date.strftime('%Y-%m-%d')
print("start_date")
print(start_date)
end_date = datetime.today() + timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')
print("end_date")
print(end_date)
data = {}
for ticker in tickers:
    yahoo_financials = YahooFinancials(ticker)
    historical_data = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
    if historical_data[ticker]:
        data[ticker] = historical_data[ticker]['prices']
dfs = []
titles = []
for ticker, prices in data.items():
    df = pd.DataFrame(prices)
    # parse_date_columns(df)
    df = df.drop('date', axis=1).set_index('formatted_date')
    df['volume'] = df['volume'].astype(float)
    # title = f'{get_coin_name(ticker[:-4])}_historical_price'
    # titles.append(title)
    dfs.append(df)
        

In [ ]:
dfs

In [ ]:
from database_handler import create_connection
from webscrape import get_last_news_date
db_session = create_connection()

get_last_news_date(db_session)

In [ ]:
start_date

In [ ]:
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d').replace(tzinfo=pytz.UTC)
start_date

In [ ]:
from webscrape import get_states_webscrape_data_from_fred_economic
from datetime import datetime
db_session = create_connection()
get_states_webscrape_data_from_fred_economic(db_session,datetime(2007,1,1),False)

In [ ]:
end_date = datetime.today()
print(end_date)
local_tz = pytz.timezone('Asia/Beirut')
utc_datetime = local_tz.localize(end_date).astimezone(pytz.UTC)
print(utc_datetime)
utc_date = utc_datetime.date()
utc_date

In [ ]:
datetime.today().date().strftime('%Y-%m-%d')

In [ ]:
from database_handler import return_query,create_connection
db_session = create_connection()
query = """SELECT EXISTS (
   SELECT 1
   FROM information_schema.tables
   WHERE table_schema = 'dw_reporting'
     AND table_name = 'dim_faang_stock_price'
);"""
return_query(db_session = db_session,query =  query)[0][0]


In [ ]:
latest_date_query = f"""
SELECT
    MAX(formatted_date)
FROM dw_reporting.dim_faang_stock_price
"""
latest_date = return_query(db_session = db_session,query = latest_date_query)[0][0]
latest_date

In [ ]:
from datetime import timedelta
latest_date+ timedelta(days=1)

In [ ]:
news_df = scrape_website(etl_date= datetime(2023, 10, 6))